In [0]:
%pip install jupytext requests
dbutils.library.restartPython()

In [0]:
import base64
import os
from pathlib import Path

import requests
import jupytext

# -------------------------
# EDIT THIS: where to save outputs in your Repo
# Example:
# TARGET_DIR = "/Workspace/Repos/<your_username>/<your_repo>/notebooks"
# or:
# TARGET_DIR = "/Workspace/Repos/<your_org>/<your_repo>/notebooks"
# -------------------------
TARGET_DIR = "/Workspace/Shared/DAMO_699-4-Capstone-Project/src/models/survival_analysis"

def _get_context():
    ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    return ctx

def _get_workspace_base_url(ctx):
    """
    Returns base URL like: https://<workspace-host>
    Uses apiUrl() when available, else falls back to browserHostName().
    """
    try:
        api_url = ctx.apiUrl().get()  # e.g. https://adb-.../api/2.0
        if "/api/" in api_url:
            return api_url.split("/api/")[0]
        return api_url
    except Exception:
        host = ctx.browserHostName().get()  # e.g. adb-...databricks.com
        return f"https://{host}"

def _get_token(ctx):
    # Works when the notebook has permission to access an API token from context
    return ctx.apiToken().get()

def export_current_notebook_to_repo(target_dir: str, fmt_py: str = "py:percent"):
    """
    Exports the *current Databricks notebook* to:
      - <target_dir>/<notebook_name>.ipynb
      - <target_dir>/<notebook_name>.py    (jupytext conversion)
    """
    ctx = _get_context()
    base_url = _get_workspace_base_url(ctx)
    token = _get_token(ctx)

    nb_path = ctx.notebookPath().get()  # workspace path of current notebook
    nb_name = Path(nb_path).name        # filename-like (no extension in Databricks)
    
    target_dir = Path(target_dir)
    target_dir.mkdir(parents=True, exist_ok=True)

    # Export from workspace as Jupyter notebook (base64 content)
    export_url = f"{base_url}/api/2.0/workspace/export"
    r = requests.get(
        export_url,
        headers={"Authorization": f"Bearer {token}"},
        params={"path": nb_path, "format": "JUPYTER"}
    )
    if r.status_code != 200:
        raise RuntimeError(f"Export failed ({r.status_code}): {r.text}")

    payload = r.json()
    ipynb_bytes = base64.b64decode(payload["content"])

    # Save .ipynb into Repo folder
    ipynb_out = target_dir / f"{nb_name}.ipynb"
    ipynb_out.write_bytes(ipynb_bytes)

    # Convert .ipynb -> .py using jupytext
    nb = jupytext.read(ipynb_out)
    py_out = target_dir / f"{nb_name}.py"
    jupytext.write(nb, py_out, fmt=fmt_py)

    print("✅ Exported and converted:")
    print(f" - IPYNB: {ipynb_out}")
    print(f" - PY:    {py_out}")
    print("\nNext: commit both files from your Repo.")

def convert_all_ipynb_to_py(target_dir: str, fmt_py: str = "py:percent", overwrite: bool = True):
    target_dir = Path(target_dir)
    count = 0

    for ipynb_path in target_dir.rglob("*.ipynb"):
        # Skip checkpoint junk if any
        if ".ipynb_checkpoints" in str(ipynb_path):
            continue

        py_path = ipynb_path.with_suffix(".py")
        if py_path.exists() and not overwrite:
            continue

        nb = jupytext.read(ipynb_path)
        jupytext.write(nb, py_path, fmt=fmt_py)

        print(f"✅ {ipynb_path.name} -> {py_path.name}")
        count += 1

    print(f"\nDone. Converted {count} notebook(s) under {target_dir}")

def convert_notebooks_in_folder(notebooks_dir: str, fmt_py: str = "py:percent"):
    notebooks_dir = Path(notebooks_dir)

    count = 0

    for ipynb_path in notebooks_dir.rglob("*.ipynb"):
        # skip checkpoints
        if ".ipynb_checkpoints" in str(ipynb_path):
            continue

        py_path = ipynb_path.with_suffix(".py")

        nb = jupytext.read(ipynb_path)
        jupytext.write(nb, py_path, fmt=fmt_py)

        print(f"Converted: {ipynb_path.name} → {py_path.name}")
        count += 1

    print(f"\nDone. Converted {count} notebook(s).")

# Run it
# export_current_notebook_to_repo(TARGET_DIR)
# convert_all_ipynb_to_py(TARGET_DIR)
convert_notebooks_in_folder(TARGET_DIR)
